# r/wallstreetbets Sentiment Analysis

In [1]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected = True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px

In [2]:
wsb = pd.read_csv("./wsbsentiment.csv", names = ['title', 'text', 'sentiment'], encoding = "utf-8", encoding_errors = 'ignore')

print(wsb.head)

<bound method NDFrame.head of                                                  title  \
0    Yolo'd my first ever 8k on an FHA 35 mortgage ...   
1    Britons use of consumer credit is rising with ...   
2                           Norwegian salmon over eggs   
3    Calling all dividend investors  Stop making in...   
4    Let get fucked This is not how imagined 2022 I...   
..                                                 ...   
347                                  we like the stock   
348                                             autist   
349                                          bagholder   
350                                            retards   
351                                      I eat crayons   

                                                  text sentiment  
0                                                  NaN  positive  
1    Excuse my retardedness but couldn't this lead ...  negative  
2                                                  NaN   neutral  
3    

In [3]:
fig = px.histogram(wsb, x = "sentiment")
fig.update_traces(marker_color="turquoise",marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5)
fig.update_layout(title_text='Sentiment Score')
fig.show()

## Naive Bayes Classifier.

Add the `punkt` tokenizing library from `nltk`.

In [4]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kim3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kim3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Remove neutral sentiment, since it kind of messes things up, then make a new list of lists with just the `title` and `sentiment` (dropping `text`).

In [5]:
wsblist = wsb.drop(columns = ['text'])
wsbtext = wsb.dropna(subset = ['text'])
wsbtext = wsbtext.drop(columns = ['title'])
wsblist = wsblist[wsblist['sentiment'] != 'neutral']
wsblist = wsblist.values.tolist()
wsbtext = wsbtext[wsbtext['sentiment'] != 'neutral']
wsbtext = wsbtext.values.tolist()
wsblist = wsblist + wsbtext

Remove stopwords and tokenize the words (painfully) (and by hand).

In [6]:
all_words = set(word.lower() for post in wsblist for word in word_tokenize(post[0]))
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append('\'s')
stopwords.append('\'d')
for stopword in stopwords:
    all_words.discard(stopword)
import pickle
words = []
try:
    with open ('words.pkl', 'rb') as fp:
        words = pickle.load(fp)
except:
    words = [({word: (word in word_tokenize(post[0])) for word in all_words}, post[1]) for post in wsblist]
    with open ('words.pkl', 'wb') as fp:
        pickle.dump(words, fp)


80/20 split on the training versus testing data.

In [7]:
print(len(words))
train_data = words[:(round(len(words) * 0.8))]
test_data = words[(round(len(words) * 0.8)):]

431


Train the Bayes classifier on the training data, then determine accuracy based on test data.

In [8]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)
print('Accuracy is ', classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(20))

Accuracy is  0.6162790697674418
Most Informative Features
                  making = True           negati : positi =      9.4 : 1.0
               contracts = True           negati : positi =      6.6 : 1.0
                  growth = True           negati : positi =      5.2 : 1.0
                    lead = True           negati : positi =      5.2 : 1.0
                  longer = True           negati : positi =      5.2 : 1.0
                       0 = True           negati : positi =      4.5 : 1.0
                   alone = True           negati : positi =      4.5 : 1.0
                  banned = True           negati : positi =      4.5 : 1.0
                dividend = True           negati : positi =      4.5 : 1.0
                 exports = True           negati : positi =      4.5 : 1.0
                   might = True           negati : positi =      4.5 : 1.0
                    rest = True           negati : positi =      4.5 : 1.0
                   seems = True           

This could probably be better with more data.

Make a custom classifier. Write a custom post in the `custom_wsb` variable, then run the cell to see what the classifier thinks the post's sentiment is.

In [9]:
from nltk.tokenize import word_tokenize
custom_wsb = "contracts"
custom_tokens = word_tokenize(custom_wsb)
print(classifier.classify(dict([token, True] for token in custom_tokens)))

negative


Try words like "bullish", "bearish", "bull", "infinity", or "hedge fund"!